#Universidad Nacional de San Antonio Abad del Cusco
Estudiante: Jhon Edwin Tacusi Larota

Codigo:     150408

Implementacion de algoritmo apriori


#ALGORITMO APRIORI

El Algoritmo Apriori trata de extraer reglas para cada posible combinación de artículos.

Para un conjunto de datos más grande, todos estos cálculos puede hacer que el proceso sea extremadamente lento. Para acelerar el proceso, debemos realizar los siguientes pasos:


*   Paso 1. Establecer un valor mínimo para el soporte y la confianza. Esto significa que solo nos interesa encontrar reglas para los elementos que tienen cierta existencia por defecto, por ejemplo, el apoyo, y tienen un valor mínimo de co-ocurrencia con otros elementos, por ejemplo, la confianza.
*   Paso 2. Extraer todos los subconjuntos que tengan un valor de soporte superior a un umbral mínimo.
*   Paso 3. Seleccionar todas las reglas de los subconjuntos con un valor de confianza superior al umbral mínimo.

*   Paso 4. Ordenar las reglas por orden descendente de lift.

#USO DE LIBRERIAS

In [1]:
import pandas as pd
import numpy as np
import itertools as it

#Modulo para obtener la frecuencia del data set


#get_frequent_itemsets

Esta función de tabla canaliza filas de tipo DM_ITEMSETS. Para obtener información sobre los tipos de datos de minería de datos y la salida canalizada de las funciones de tabla,

playlist:el dataset de las transacciones  lista de listas.

min_support:soporte minimo int

total_Itemsets:La funcion devuelve lista de listas.


In [2]:
def get_frequent_itemsets(playlists, min_support=0.2):
    #CREAMOS 2 ARREGLOS
    total_Itemsets = []
    total_valores = []
    num_iter=1
    #ORDENAMOS EL PLAYLIST
    print('Calculando itemsets de long %2i'%(num_iter))
    #OBTENEMOS LO ITEMSET
    itemset = []
    val_item = []
    for miniset in playlists:
        for item in miniset:
            item = str(item)
            if(item in itemset):
                val_item[itemset.index(item)]+=1
            else:
                itemset.append(item)
                val_item.append(1)
    #NORMALIZAMOS LOS VALORES DE 0 A 1 LOS VALORES ACUMULADOS
    long_playlists = len(playlists)
    val_item= [x/long_playlists for x in val_item]
    #ELIMINAMOS TODOS LOS VALORES QUE NO PASEN EL UMBRAL "min_support"
    valid_itemset,valid_values= LimpiarItemset(itemset,val_item,min_support)
    #ORDENAMOS LO VALORES
    temp_duplas = dict(zip(valid_itemset,valid_values)).items()
    valid_itemset = []
    valid_values = []
    for dupla in sorted(temp_duplas):
        valid_itemset.append(dupla[0])
        valid_values.append(dupla[1])
    #PROCEDEMOS A INICIAR UN CICLO PARA CALCULAR ITEMSETS DE MAYOR CARDINALIDAD
    last_itemset = [[x] for x in valid_itemset]
    print("└──Terminado || cant. elementos (%i)"%(len(last_itemset)))
    #COMPROBAMOS SI LS ULTIMA ITERACION TIENE ITEMS VALIDOS
    while(True):
        if(len(last_itemset)==0):
            break
        total_Itemsets.append(last_itemset)
        total_valores.append(valid_values)
        if(len(last_itemset)==1):
            break    
        num_iter+=1
        print('Calculando itemsets de long %2i'%(num_iter))
        Pos_Itemset = GenerarPosibleItemset(last_itemset)
        Val_PosItems = ContarAparicionItemSet(Pos_Itemset,playlists)                 
        #NORMALIZAMOS LOS VALORES DE 0 A 1 LOS VALORES ACUMULADOS
        Val_PosItems= [x/long_playlists for x in Val_PosItems]
        #ELIMINAMOS TODOS LOS VALORES QUE NO PASEN EL UMBRAL "min_support"
        last_itemset, valid_values = LimpiarItemset(Pos_Itemset,Val_PosItems,min_support)
        print("└──Terminado || cant. elementos (%i)"%(len(last_itemset)))
    return total_Itemsets,total_valores

#Modulo para obtener la asociaciones del data set

#generate_association_rules

La generación de reglas es una tarea común en la extracción de patrones frecuentes. Una regla de asociación es una expresión de implicación de la formaX→ Y, dónde X y Yson conjuntos de elementos separados 

frequent_itemsets:Itemset obtenidos de get_frequent_itemsets lista de lista de str


confidence:confianza minima int

lift:suntento minimo int

In [3]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift =0):
    All_rules = []
    All_lift = []
    All_confidence = []
    All_support = []
    Long_regla_actual = 2
    itemsets = frequent_itemsets[0]
    itemsvalues = frequent_itemsets[1]
    Xs = []
    Ys = []
    print('Calculas todas las posibles reglas')
    #SE OBTENDRA PARTELAS REGLAS
    while(Long_regla_actual<=len(itemsets)):
        itemset_actual = itemsets[Long_regla_actual-1]
        for item in itemset_actual:
            xs,ys = ObtenerPosiblesReglas(item)
            Xs+=xs
            Ys+=ys
        Long_regla_actual+=1
    print('Limpieza de reglas repetidas')
    #UNA VE TENIENDO TODAS LA REGLAS DE LA FORMA  X->Y ELIMINAMOS LAS REPETIDAS
    Xs,Ys = LimpiarReglasRepetidas(Xs,Ys)
    print('Calculando lift y confianza')
    # SE REALIZA EL CALCULO DE lift Y confianza
    for k in range(len(Xs)):
        x = Xs[k]
        y = Ys[k]
        xuy = x+y
        xuy.sort()
        pos = itemsets[len(x)-1].index(x)
        Px = itemsvalues[len(x)-1][pos]
        pos = itemsets[len(y)-1].index(y)
        Py = itemsvalues[len(y)-1][pos]
        pos = itemsets[len(xuy)-1].index(xuy)
        Pxuy = itemsvalues[len(xuy)-1][pos]
        #Calculamos la confianza y el lift
        confianza_act = Pxuy/Px
        lift_act = Pxuy/Px/Py
        #comprobamos si pasa el umbral
        if(lift_act >= lift and confianza_act >= confidence):
            All_rules.append([x,y])
            All_confidence.append(confianza_act)
            All_lift.append(lift_act)
            All_support.append(Pxuy)
    return  All_rules,All_confidence,All_lift,All_support

#Modulo para Generar posibles  itemset



In [4]:
#GENERAMOS LOS POSIBLES ITEMSETS
def GenerarPosibleItemset(Old_Itemset):
    PosItemSet = []
    #REALIZAMOS LA OPERACION A u B SOBRE LOS ITEMS SI EL ITEM SOLO TIENE 1 ELEMENYO MAS QUE SUS PADRES SE LE CONSIDERA COMO ITEMSET
    for k in range(0,len(Old_Itemset)-1):
        for j in range(k+1,len(Old_Itemset)):
            Posible_Item = sorted(list(set(Old_Itemset[k]+Old_Itemset[j])))
            if(len(Posible_Item)==len(Old_Itemset[0])+1 
               and Posible_Item not in PosItemSet):
                PosItemSet.append(Posible_Item)
    return PosItemSet

#Modulo para  contar apariciones de Itemsets

In [5]:
#CONTAR APARICIONES DE ITEMSET
def ContarAparicionItemSet(Pos_Itemset, Dataset):
#CREAMOS UN CONTADOR
    Contador_Tot = [0]*len(Pos_Itemset)
    #INICIAMOS CON EL PROCESO DE CONTAR 
    for k in range(len(Pos_Itemset)):       
        Item = Pos_Itemset[k]
        #RECORREMOS EL DATA SET
        for tupla in Dataset:
          #REALIZAMOS LA OPERACION Item U Tupla
            tupla = set(tupla)
            temp = list(set(Item).union(tupla))
            tupla = list(tupla)
            if(len(temp)==len(tupla)):
                Contador_Tot[k]+=1
    return Contador_Tot

#modulo para limpiar Itemset 

In [6]:
#FUNCION QUE SE ENCARGA DE ELIMINAR VALORES DE ITEMSET DONDE LA CONFIANZA NO SUPERE EL UMBRAL
def LimpiarItemset(Itemset,Valores,umbral):
    items_validos = []
    for valor_act in Valores:
        items_validos.append(valor_act>=umbral)
    Itemset_limpio = list(it.compress(Itemset,items_validos))
    Valores_limpio = list(it.compress(Valores,items_validos))
    return Itemset_limpio,Valores_limpio

#Modulo para Obtener las posibles reglas

In [7]:
#OBTENEMOS LA REGLAS POSIBLES
def ObtenerPosiblesReglas(item):
    combinaciones = list(it.permutations(item))
    Xs_actuales = []
    Ys_actuales = []
    for tupla in combinaciones:
        tupla = list(tupla)
        for k in range(1,len(tupla)):
            #Basandonos en que las reglas son X -> Y los separamos para en 2 variables
            X, Y = tupla[:k],tupla[k:]
            X.sort() 
            Y.sort()
            Xs_actuales.append(X)
            Ys_actuales.append(Y)
    return Xs_actuales,Ys_actuales

# Lipiamos las reglas repetidas 

In [8]:
#ELIMINAMOS LAS REGLAS REPETIDAS
def LimpiarReglasRepetidas(X,Y):
    validos = [True]*len(X)
    #SI EL VALOR ACTUAL ES VALIDO COMPROBAMOS SI TIENE UN REPETIDO
    for k in range(len(X)-1):
        if(validos[k]):
            for j in range(k+1,len(X)):

                if(validos[j] and 
                    X[k] == X[j] and
                    Y[k] == Y[j]):
                    validos[j]=False
    #ELIMINAMOS LOS NO VALIDOS
    X = list(it.compress(X,validos))
    Y = list(it.compress(Y,validos))
    return X,Y

# Mostrar las reglas despues que ya fueron limpiadas

In [9]:
def MostrarReglas(Xs,Ys):
    for k in range(len(Xs)):
        print(f'{Xs[k]} -> {Ys[k]}')

#EJECUCION

Lectura de la base de datos .npy

In [10]:
datos= np.load('spotify.npy',allow_pickle=True)
X = datos.tolist()
spotify_playlists = [X[k] for k in X]
total_Itemsets,total_valores = get_frequent_itemsets(spotify_playlists, 0.02)

Calculando itemsets de long  1
└──Terminado || cant. elementos (161)
Calculando itemsets de long  2
└──Terminado || cant. elementos (3)
Calculando itemsets de long  3
└──Terminado || cant. elementos (0)


In [11]:
print(f'Longitud máxima encontrada: {len(total_Itemsets)}')
# MOSTRAR ITEMSETS VALIDOS ENCONTRADOS
for k in range(len(total_Itemsets)):
    print('===== %3i ======='%(k+1))
    for j in range(len(total_Itemsets[k])):
        print('%s || %3.2f'%(total_Itemsets[k][j],total_valores[k][j]*100))

Longitud máxima encontrada: 2
=====   1 =======
['1-800-273-8255'] || 2.64
['679 (feat. Remy Boyz)'] || 2.46
['Again'] || 2.14
["Ain't No Mountain High Enough"] || 2.33
['Alive'] || 2.33
['All I Want'] || 2.34
['All of Me'] || 2.41
['Alone'] || 2.15
['Alright'] || 2.81
['Angel'] || 2.25
['Antidote'] || 2.64
['Bad and Boujee (feat. Lil Uzi Vert)'] || 3.53
['Bank Account'] || 2.31
['Beautiful'] || 2.02
['Believe'] || 2.20
['Believer'] || 2.54
['Black Beatles'] || 2.96
['Blessings'] || 2.84
['Body Like A Back Road'] || 2.73
['Bohemian Rhapsody - Remastered 2011'] || 2.43
['Bounce Back'] || 3.00
['Broccoli (feat. Lil Yachty)'] || 4.02
['CAN\'T STOP THE FEELING! (Original Song from DreamWorks Animation\'s "TROLLS")'] || 2.09
["Can't Feel My Face"] || 2.26
['Caroline'] || 3.95
['Cheap Thrills'] || 2.14
['Chicken Fried'] || 2.18
['Chill Bill'] || 2.30
['Closer'] || 7.61
['Cold Water (feat. Justin Bieber & MØ)'] || 2.41
['Come On Eileen'] || 2.07
['Congratulations'] || 4.10
['Controlla'] || 2.

In [14]:
Reglas,Confianza,Lift,Soporte = generate_association_rules([total_Itemsets,total_valores],0.28,2.65)

Calculas todas las posibles reglas
Limpieza de reglas repetidas
Calculando lift y confianza


In [15]:
for k in range(len(Reglas)):
    print(f"{k+1} ======")
    print(f'{Reglas[k][0]} -> {Reglas[k][1]}')
    print(f'soporte: {Soporte[k]*100}')
    print(f'confianza: {Confianza[k]*100}')
    print(f'lift: {Lift[k]}')

1 ======
['Congratulations'] -> ['HUMBLE.']
soporte: 2.1399999999999997
confianza: 52.195121951219505
lift: 11.10534509600415
2 ======
['HUMBLE.'] -> ['Congratulations']
soporte: 2.1399999999999997
confianza: 45.53191489361702
lift: 11.10534509600415
3 ======
['HUMBLE.'] -> ['Mask Off']
soporte: 2.04
confianza: 43.40425531914894
lift: 13.735523835173714
4 ======
['Mask Off'] -> ['HUMBLE.']
soporte: 2.04
confianza: 64.55696202531645
lift: 13.735523835173714
5 ======
['HUMBLE.'] -> ['XO TOUR Llif3']
soporte: 2.04
confianza: 43.40425531914894
lift: 13.152804642166345
6 ======
['XO TOUR Llif3'] -> ['HUMBLE.']
soporte: 2.04
confianza: 61.81818181818181
lift: 13.152804642166345


#analizando las reglas se obtuvo las siguientes  canciones  

* Congratulations
* HUMBLE
* Mask Off
* XO TOUR Llif3

# frente a los datos obtenidos podemos apreciar que:

+ Observando las reglas podemos ver que las 4 canciones  son del mismo genero.
+ observando se puede determinar que 3 canciones son del 2017 y 1 es del 2016
+ Todos los artistas son hombres
+ La artista de la cancion HUMBLEE colaboro con todos los demas artistas
